# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116703e+02     1.640322e+00
 * time: 0.08267498016357422
     1     1.061533e+01     8.991512e-01
 * time: 1.2896251678466797
     2    -1.107152e+01     1.044894e+00
 * time: 1.4031860828399658
     3    -3.400593e+01     8.196776e-01
 * time: 1.5806269645690918
     4    -4.731192e+01     6.154606e-01
 * time: 1.7121789455413818
     5    -5.672497e+01     1.974289e-01
 * time: 1.8582940101623535
     6    -5.979224e+01     1.274506e-01
 * time: 1.986152172088623
     7    -6.092150e+01     5.490411e-02
 * time: 2.0887889862060547
     8    -6.132677e+01     7.134503e-02
 * time: 2.1912829875946045
     9    -6.164476e+01     3.374633e-02
 * time: 2.295959949493408
    10    -6.186621e+01     3.051725e-02
 * time: 2.4009389877319336
    11    -6.203862e+01     2.297740e-02
 * time: 2.5171711444854736
    12    -6.210336e+01     1.575298e-02
 * time: 2.620687961578369
    13    -6.216607e+01     1.393041e-02
 * time: 2.724204063415

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557676
    AtomicNonlocal      14.8522647
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666461144
